#  Homework 8 - Berkeley STAT 157

**Your name: XX, SID YY, teammates A,B,C** (Please add your name, SID and teammates to ease Ryan and Rachel to grade.)

**Please submit your homework through [gradescope](http://gradescope.com/)**

Handout 4/9/2019, due 4/16/2019 by 4pm.

This homework deals with sequence models for text and numbers. Due to the computational cost, we strongly encourage you to implement this on a GPU enabled machine. To make things a bit more interesting we will use a larger text collection here - [Shakespeare's collected works](http://www.gutenberg.org/files/100/100-0.txt) which are freely downloadable at Project Gutenberg. 

**This is teamwork.**

## Prerequisites - Load Data

In [1]:
import urllib3
import collections
import re
shakespeare = 'http://www.gutenberg.org/files/100/100-0.txt'

http = urllib3.PoolManager()
text = http.request('GET', shakespeare).data.decode('utf-8')
raw_dataset = ' '.join(re.sub('[^A-Za-z]+', ' ', text).lower().split())

print('number of characters: ', len(raw_dataset))
print(raw_dataset[0:70])

number of characters:  5032359
project gutenberg s the complete works of william shakespeare by willi


This dataset is quite a bit bigger than the time machine (5 million vs. 160k). For convenience we also include the remaining preprocessing steps. A bigger dataset will allow us to generate more meaningful models. 

In [3]:
idx_to_char = list(set(raw_dataset))
char_to_idx = dict([(char, i) for i, char in enumerate(idx_to_char)])
vocab_size = len(char_to_idx)
corpus_indices = [char_to_idx[char] for char in raw_dataset]
sample = corpus_indices[:20]
print('chars:', ''.join([idx_to_char[idx] for idx in sample]))
print('indices:', sample)

train_indices = corpus_indices[:-100000]
test_indices = corpus_indices[-100000:]

chars: project gutenberg s 
indices: [21, 12, 3, 11, 7, 23, 9, 10, 17, 4, 9, 7, 19, 22, 7, 12, 17, 10, 24, 10]


In [4]:
print(vocab_size)
print(idx_to_char)
print(char_to_idx)
print(len(corpus_indices))

27
['k', 'y', 'w', 'o', 'u', 'i', 'a', 'e', 'q', 't', ' ', 'j', 'r', 'v', 'f', 'z', 'm', 'g', 'd', 'n', 'x', 'p', 'b', 'c', 's', 'h', 'l']
{'k': 0, 'y': 1, 'w': 2, 'o': 3, 'u': 4, 'i': 5, 'a': 6, 'e': 7, 'q': 8, 't': 9, ' ': 10, 'j': 11, 'r': 12, 'v': 13, 'f': 14, 'z': 15, 'm': 16, 'g': 17, 'd': 18, 'n': 19, 'x': 20, 'p': 21, 'b': 22, 'c': 23, 's': 24, 'h': 25, 'l': 26}
5032359


Lastly we import other useful libraries to help you getting started.

In [5]:
import d2l
import math
from mxnet import autograd, gluon, init, nd
from mxnet.gluon import loss as gloss, nn, rnn
import time
import mxnet

## 3. $N$-Gram Model

So far we only considered latent variable models. Let's see what happens if we use a regular $N$-gram model and an autoregressive setting. That is, we aim to predict the next character given the current characters one character at a time. For this implement the following:

1. Split data into $(x,y)$ pairs as before, just that we now use very short subsequences, e.g. only $5$ characters. That is, `But Brutus` turns into the tuples (`(But B, r)`, `(ut Br, u)`, `(t Bru, t)`, `( Brut, u)`, `(Brutu, s)`). 
1. Use one-hot encoding for each character separately and combine them all. 
    * In one case use a sequential encoding to obtain an embedding proportional to the length of the sequence.
    * Use a bag of characters encoding that sums over all occurrences.
1. Implement an MLP with one hidden layer and 256 hidden units.
1. Train it to output the next character.

How accurate is the model? How does the number of operations and weights compare to an RNN, a GRU and an LSTM discussed above?

## Training Dataset

In [6]:
ctx = d2l.try_gpu()
embedding = 5 # embedding dimension for autoregressive model
T = len(raw_dataset)
features = nd.zeros((T-embedding, embedding), ctx = ctx)

In [207]:
for i in range(embedding):
    features[:,i] = corpus_indices[i:T-embedding+i]
labels = nd.array(corpus_indices[embedding:],ctx =ctx).reshape(-1,1)

In [208]:
features


[[20.  6.  7. 18. 14.]
 [ 6.  7. 18. 14. 12.]
 [ 7. 18. 14. 12. 15.]
 ...
 [22.  9. 14.  4.  7.]
 [ 9. 14.  4.  7.  7.]
 [14.  4.  7.  7. 21.]]
<NDArray 5032354x5 @cpu(0)>

In [209]:
labels


[[12.]
 [15.]
 [ 9.]
 ...
 [ 7.]
 [21.]
 [ 0.]]
<NDArray 5032354x1 @cpu(0)>

### Prediction Dataset

In [7]:
predict_dataset = 'but brutus is an honorable man'
corpus_indices_predict = [char_to_idx[char] for char in predict_dataset]
T_predict = len(predict_dataset)
predict_features = nd.zeros((T_predict-embedding, embedding),ctx = ctx)

In [8]:
for i in range(embedding):
    predict_features[:,i] = corpus_indices_predict[i:T_predict-embedding+i]
predict_labels = nd.array(corpus_indices_predict[embedding:],ctx = ctx).reshape(-1,1)

In [9]:
predict_features


[[22.  4.  9. 10. 22.]
 [ 4.  9. 10. 22. 12.]
 [ 9. 10. 22. 12.  4.]
 [10. 22. 12.  4.  9.]
 [22. 12.  4.  9.  4.]
 [12.  4.  9.  4. 24.]
 [ 4.  9.  4. 24. 10.]
 [ 9.  4. 24. 10.  5.]
 [ 4. 24. 10.  5. 24.]
 [24. 10.  5. 24. 10.]
 [10.  5. 24. 10.  6.]
 [ 5. 24. 10.  6. 19.]
 [24. 10.  6. 19. 10.]
 [10.  6. 19. 10. 25.]
 [ 6. 19. 10. 25.  3.]
 [19. 10. 25.  3. 19.]
 [10. 25.  3. 19.  3.]
 [25.  3. 19.  3. 12.]
 [ 3. 19.  3. 12.  6.]
 [19.  3. 12.  6. 22.]
 [ 3. 12.  6. 22. 26.]
 [12.  6. 22. 26.  7.]
 [ 6. 22. 26.  7. 10.]
 [22. 26.  7. 10. 16.]
 [26.  7. 10. 16.  6.]]
<NDArray 25x5 @gpu(0)>

In [10]:
predict_labels


[[12.]
 [ 4.]
 [ 9.]
 [ 4.]
 [24.]
 [10.]
 [ 5.]
 [24.]
 [10.]
 [ 6.]
 [19.]
 [10.]
 [25.]
 [ 3.]
 [19.]
 [ 3.]
 [12.]
 [ 6.]
 [22.]
 [26.]
 [ 7.]
 [10.]
 [16.]
 [ 6.]
 [19.]]
<NDArray 25x1 @gpu(0)>

## Encoding

In [11]:
def to_onehot_length(X, vocab_size):
    inputs = [nd.one_hot(x, vocab_size) for x in X]
    matrix_size = inputs[0].size
    for i in range(len(inputs)):
        inputs[i] = inputs[i].reshape(1,matrix_size)
    return inputs

In [12]:
#def to_onehot_output(Y, vocab_size):
    #outputs = [nd.one_hot(y, vocab_size) for y in Y]
    #return outputs

In [13]:
def to_onehot_sum(X, vocab_size):
    inputs = [nd.one_hot(x, vocab_size) for x in X]
    matrix_size = inputs[0].size
    for i in range(len(inputs)):
        inputs[i] = nd.sum(inputs[i],axis = 0).reshape(1, vocab_size)
    return inputs

In [ ]:
#这俩不能一起跑，内存会炸
inputs = to_onehot_length(features, vocab_size)

In [157]:
inputs_sum = to_onehot_sum(features, vocab_size)

In [14]:
#predictIn = to_onehot_length(predict_features, vocab_size)
#print('finished')
#predictInSum = to_onehot_sum(predict_features, vocab_size)
#print('finished')

finished
finished


### MLP

In [230]:
net = gluon.nn.Sequential()
net.add(gluon.nn.Dense(256, activation='relu'))
net.add(gluon.nn.Dense(vocab_size))
net.initialize(init.Xavier())

loss = gloss.SoftmaxCrossEntropyLoss()

In [ ]:
ntrain = 900000
train_data = gluon.data.ArrayDataset(inputs[:ntrain], labels[:ntrain])
test_data  = gluon.data.ArrayDataset(inputs[ntrain:1000000], labels[ntrain:1000000])

batch_size = 30
trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': 0.8})
num_epochs = 5 

train_iter = gluon.data.DataLoader(train_data, batch_size, shuffle=True)
test_iter = gluon.data.DataLoader(test_data, batch_size, shuffle=True)

In [ ]:
#for second encoding version
ntrain = 900000
train_data_2 = gluon.data.ArrayDataset(inputs_sum[:ntrain], labels[:ntrain])
test_data_2  = gluon.data.ArrayDataset(inputs_sum[ntrain:1000000], labels[ntrain:1000000])

batch_size = 30
trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': 0.5})
num_epochs = 5

train_iter_2 = gluon.data.DataLoader(train_data_2, batch_size, shuffle=True)
test_iter_2 = gluon.data.DataLoader(test_data_2, batch_size, shuffle=True)

In [ ]:
d2l.train_ch5(net, train_iter, test_iter, batch_size, ctx, num_epochs)

In [15]:
# This function is saved in the d2l package for future use
def predict_ngram(prefix, num_chars, net, params, features,
                num_hiddens, vocab_size, ctx):
    
    output = [char_to_idx[prefix[0]]]
    
    for t in range(num_chars + len(prefix) - 1):
        # The output of the previous time step is taken as the input of the
        # current time step.
        
        # Calculate the output and update the hidden state
        Y = net(X)
        # The input to the next time step is the character in the prefix or
        # the current best predicted character
        if t < len(prefix) - 1:
            # Read off from the given sequence of characters
            output.append(char_to_idx[prefix[t + 1]])
        else:
            # This is maximum likelihood decoding. Modify this if you want
            # use sampling, beam search or beam sampling for better sequences.
            output.append(int(Y[0].argmax(axis=1).asscalar()))
            
    return ''.join([idx_to_char[i] for i in output])

In [17]:
prefix = 'but brutus is an honorable man'
output = [char_to_idx[prefix[0]]]

In [18]:
output

[22]